# Time Headway Calculator

The goal of this notebook is to establish time headways for individual following events, as a histogram based on the speed of the following vehicle.

Authors: Sally Kim, Jonathan Sprinkle

In [10]:
import signal
import pandas as pd
import sys, math, time, datetime
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib import style
import numpy as np
from matplotlib.pyplot import cm
import pickle
from strym import strymread
import strym
import copy

In [18]:
# change this to your data folder
# datafolder='/Users/sallykim/Desktop/2021_08_09/'
datafolder='/Users/sprinkle/work/data/sally/2021_08_09/'

import glob
csvlist = glob.glob(datafolder+"*2T3*CAN*.csv")

num_of_files = len(csvlist)
print("Total number of datafiles in {} is {}.".format(datafolder, num_of_files))
# dbcfile = '../examples/newToyotacode.dbc'

Total number of datafiles in /Users/sprinkle/work/data/sally/2021_08_09/ is 3.


In [19]:
# choose the first file, and then plot the speed, as well as the lead vehicle distance
# Look in Minitest 1 or in 

#dbcfile_17 = '/Users/sallykim/opt/miniconda3/lib/python3.9/site-packages/strym/dbc/toyota_rav4_hybrid.dbc'
datafile_17 = datafolder+'2021-08-09-17-42-45_2T3Y1RFV8KC014025_CAN_Messages.csv'
#data_17 = pd.read_csv(datafile_17)


In [20]:
#db_17 = str.initializeDBC_Cantools(dbcfile_17)
#speed_17 = str.convertData("SPEED", "SPEED", data_17, db_17)
#plt.plot(speed_17.Time, speed_17.Message)
#plt.title("08/09/2021 drive, file 1")
#plt.xlabel("time")
#plt.ylabel("speed")

In [ ]:
r0 = strymread(datafile_17)
speed = r0.speed()
try:
    speed_17 = r0.get_ts('SPEED','SPEED')
    long_dist_17 = r0.get_ts('LEAD_INFO','LEAD_LONG_DIST')
    long_dist_17_2 = r0.lead_distance()
except:
    print('Oops, a specific error message Sally writes')

[2022_07_18_12_51_47] (root) INFO: Vehicle model infered is toyota-rav4-2019


In [ ]:
# choose the first file, and then plot the speed, as well as the lead vehicle distance
# Look in Minitest 1 or in 
fig, ax = strymread.create_fig(2)
ax[0].scatter(x=speed_17['Time'] - speed_17['Time'][0],y=speed_17['Message'])
ax[0].set(xlabel='Time (s) from start of drive',ylabel='Speed (km/hr)',title='Speed vs. Time')
ax[1].scatter(x=long_dist_17_2['Time'] - long_dist_17_2['Time'][0],y=long_dist_17_2F['Message'])
ax[1].set(xlabel='Time (s) from start of drive',ylabel='Lead vehicle longitudinal distance (m)',title='Lead Distance vs. Time')

In [ ]:
# find all times where the lead vehicle distance changes by more than 5


In [ ]:
# remove all times where the lead vehicle distance is > 200